In [3]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import nrrd
print("------------------------------------------------------------------------------------------------")
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print("------------------------------------------------------------------------------------------------")

2024-07-07 10:42:00.466509: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-07 10:42:01.114440: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


------------------------------------------------------------------------------------------------
2.16.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
------------------------------------------------------------------------------------------------


In [4]:

def encoder_block(inputs, output_channels, lastlayer=False):
    """
    Two 3x3x3 convolutions with batch normalization and ReLU activation
    2x2x2 max pool
    """

    # 3x3x3 convolutions with ReLU activation
    x = tf.keras.layers.Conv3D(int(output_channels/2), kernel_size=3, strides=1, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv3D(output_channels, kernel_size=3, strides=1, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # 2x2x2 max pool

    if not lastlayer:
        x_maxPool = tf.keras.layers.MaxPool3D(pool_size=2, strides=2, padding = 'same')(x)
    else:
        x_maxPool = x

    return x, x_maxPool

def decoder_block(inputs, skip_features, output_channels):

    # Upsampling with 2x2x2 filter
    x = tf.keras.layers.Conv3DTranspose(output_channels*2, kernel_size=2, strides=2, padding = 'same')(inputs)

# Concatenate the skip features
    x = tf.keras.layers.Concatenate()([x, skip_features])

    # 2 convolutions with 3x3 filter, batch normalization, ReLU activation
    x = tf.keras.layers.Conv3D(output_channels, kernel_size=3, strides=1, padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv3D(output_channels, kernel_size=3, strides=1, padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    return x

def unet_3D():
    inputs = tf.keras.Input(shape=(64, 64, 64, 1,))

    e1_skip, e1_maxpool = encoder_block(inputs, 64)
    e2_skip, e2_maxpool = encoder_block(e1_maxpool, 128)
    e3_skip, e3_maxpool = encoder_block(e2_maxpool, 256)
    _, e4 = encoder_block(e3_maxpool, 512, True)

    decoder1 = decoder_block(e4, e3_skip, 256)
    decoder2 = decoder_block(decoder1, e2_skip, 128)
    decoder3 = decoder_block(decoder2, e1_skip, 64)

    outputs = tf.keras.layers.Conv3D(1, 1, strides = 1)(decoder3)
    # outputs = tf.keras.layers.Conv3D(2, 1, strides = 1)(decoder3)
    # outputs = tf.keras.layers.Reshape((64*64*64, 2))(outputs)
    outputs = tf.keras.layers.Activation('sigmoid')(outputs)

    model = tf.keras.models.Model(inputs = inputs,  outputs = outputs,  name = 'Unet3D')

    return model
    

In [5]:
def iou(y_true, y_pred, smooth=0.000000001):
    # yt = K.argmax(y_true, axis=2)
    # yp = K.argmax(y_pred, axis=2)
    # print(y_pred)
    #yp = y_pred[0]
    # yp[yp>=0.5]=1
    # yp[yp<0.5]=0
    yp = y_pred
    yp = tf.where(yp >= 0.5, tf.ones_like(yp), yp)
    yp = tf.where(yp < 0.5, tf.zeros_like(yp), yp)
    yp = K.cast(yp, np.float32)

    yt = K.cast(y_true, np.float32)

    # print(yt.shape)
    # print(yp.shape)
    
    intersection = K.sum(yt * yp)
    union = K.sum(yt) + K.sum(yp)
    # intersection = K.sum(yt * yp, axis=1)
    # union = K.sum(yt, axis=1) + K.sum(yp, axis=1)
    return (intersection + smooth) / (union-intersection+smooth)

    
    

In [6]:
model = unet_3D()
# model.summary()

print("compiling model")
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='dice')#, metrics=[iou])

2024-07-07 10:42:01.837015: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1114 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti, pci bus id: 0000:65:00.0, compute capability: 8.9


compiling model


In [7]:
import os
import matplotlib.pyplot as plt

In [8]:
import json
# Get Patch Coordinates

with open("patch_coords.json", 'r') as f:
    p_coords = json.load(f)

print(p_coords)

[[512, 512, 211], [1, 311, 375, 179, 243, 27, 91], [2, 243, 307, 139, 203, 33, 97], [3, 209, 273, 143, 207, 40, 104], [4, 188, 252, 175, 239, 54, 118], [5, 220, 284, 175, 239, 66, 130], [6, 168, 232, 137, 201, 88, 152], [7, 311, 375, 145, 209, 90, 154], [8, 152, 216, 172, 236, 103, 167], [9, 186, 250, 175, 239, 118, 182], [10, 124, 188, 204, 268, 121, 185], [11, 328, 392, 177, 241, 122, 186], [12, 276, 340, 162, 226, 122, 186], [13, 173, 237, 207, 271, 129, 193]]


In [9]:
cp_number = "{:04d}".format(140)
print(cp_number)
model.load_weights("./checkpoints/cp-0140.weights.h5")

number = 18
reformation = np.zeros(shape=(p_coords[0]))

for patch in range(1, len(p_coords)):
    print(p_coords[patch])    
    try:
        X, _ = nrrd.read("./test_data/inputs/18_volume_"+str(p_coords[patch][0])+".nrrd")
        X = np.array([X]).astype(np.float32)
        X = np.expand_dims(X, -1)
        
        y = model.predict(X, verbose=0)
        y = y[0]
        y[y>=0.5]=1
        y[y<0.5]=0
        y = y[...,0]
        print(y.shape)
        reformation[p_coords[patch][1]:p_coords[patch][2], p_coords[patch][3]:p_coords[patch][4], p_coords[patch][5]:p_coords[patch][6]] += y
    except:
        pass
        
reformation[reformation > 1] = 1

nrrd.write("./test_data/reform/predictions/"+str(number)+"_checkpoint_"+str(cp_number)+".nrrd", reformation)



0140


/home/kevin.wang/miniconda3/envs/cactas/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 130 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


[1, 311, 375, 179, 243, 27, 91]
[2, 243, 307, 139, 203, 33, 97]
[3, 209, 273, 143, 207, 40, 104]
[4, 188, 252, 175, 239, 54, 118]
[5, 220, 284, 175, 239, 66, 130]
[6, 168, 232, 137, 201, 88, 152]
[7, 311, 375, 145, 209, 90, 154]
[8, 152, 216, 172, 236, 103, 167]
[9, 186, 250, 175, 239, 118, 182]
[10, 124, 188, 204, 268, 121, 185]
[11, 328, 392, 177, 241, 122, 186]
[12, 276, 340, 162, 226, 122, 186]
[13, 173, 237, 207, 271, 129, 193]


In [8]:
np.unique(reformation)

array([0., 1.])

In [10]:
def compute_iou(y_true, y_pred, smooth=0.000000001):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred)
    return (intersection + smooth) / (union-intersection+smooth)
    
def f1_sens_prec(y_true, y_pred, smooth=0.000000001):
    ones_matrix = np.ones(shape=(np.shape(y_true)))
    yp_negatives = ones_matrix - y_pred
    yt_negatives = ones_matrix - y_true
    
    tp = np.sum(y_true * y_pred)
    fp = np.count_nonzero((y_pred-y_true) == 1)
    tn = np.sum(yt_negatives * yp_negatives)
    fn = np.count_nonzero((yp_negatives-yt_negatives) == 1)

    fscore = (2 * tp) / (2*tp + fp + fn)
    precision = tp / (tp + fp)
    sensitivity = tp / (tp + fn)

    return fscore, sensitivity, precision

In [11]:
gt, _ = nrrd.read("./test_data/reform/predictions/"+str(number)+"_gt.nrrd")

FileNotFoundError: [Errno 2] No such file or directory: './test_data/reform/predictions/18_gt.nrrd'

In [17]:
compute_iou(gt, reformation)
f1_sens_prec(gt, reformation)

(0.6236559139784946, 0.5237020316027088, 0.770764119601329)

In [2]:
number = 18

metrics = []

gt, _ = nrrd.read("./test_data/reform/predictions/"+str(number)+"_gt.nrrd")

best_cp = 0
best_iou = 0

for cp in range(1, 10):
    cp_number = "{:04d}".format(cp)
    print(cp_number)
    model.load_weights("./checkpoints/cp-"+str(cp_number)+".weights.h5")
    
    reformation = np.zeros(shape=(p_coords[0]))
    
    for patch in range(1, len(p_coords)): 
        try:
            X, _ = nrrd.read("./test_data/inputs/18_volume_"+str(p_coords[patch][0])+".nrrd")
            X = np.array([X]).astype(np.float32)
            X = np.expand_dims(X, -1)
            
            y = model.predict(X, verbose=0)
            y = y[0]
            y[y>=0.5]=1
            y[y<0.5]=0
            y = y[...,0]
            reformation[p_coords[patch][1]:p_coords[patch][2], p_coords[patch][3]:p_coords[patch][4], p_coords[patch][5]:p_coords[patch][6]] += y
        except:
            pass
            
    reformation[reformation > 1] = 1
    
    nrrd.write("./test_data/reform/predictions/"+str(number)+"_checkpoint_"+str(cp_number)+".nrrd", reformation)
    iou = compute_iou(gt, reformation)
    f1, sens, prec = f1_sens_prec(gt, reformation)
    if iou > best_iou:
        best_iou = iou
        iou_other_metrics = [iou, f1, sens, prec]
        best_cp = cp
    print("IOU: " + str(iou) + ", F1: "+ str(f1) + ", Sensitivity: " + str(sens) + ", Precision: " + str(prec))
    metrics.append([iou, f1, sens, prec])

metrics_path = "./test_data/reform/metrics/"+str(number)+".csv"
numpy.savetxt("data3.csv", np.array(metrics),  delimiter = ",")

NameError: name 'nrrd' is not defined

In [20]:
print(iou_other_metrics, best_cp)

[0.5712971481144699, 0.7271662763466042, 0.7009029345372461, 0.7554744525547445] 62


In [ ]:
n

In [12]:
metrics = []
metrics.append([1,2,3])
metrics.append([4,5,6])


In [18]:
with open("test.csv", "r") as f:
    print(f.read())

1,2,3,4,5,6
